In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

secret_label = "Hugging_Face"
hf_token = UserSecretsClient().get_secret(secret_label)
login(hf_token)



In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "mistralai/Mistral-7B-Instruct-v0.1" 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)




2025-07-07 09:14:29.139908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751879669.570688      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751879669.696033      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
import random
def generate_personalized_example(word, pos, user_data, language):
    name = user_data.get("name", "User")
    level = user_data.get("level", "intermediate")
    topics = user_data.get("topics", ["general"])

    topic = random.choice(topics)

    prompt = f"""
You are an {language} tutor for language learners. The student is {name}, an {level} learner. 

Generate ONE simple {language} sentence using the word "{word}" ({pos}). The sentence should relate to the topic "{topic}", and be suitable for {level} level. 

Only output the sentence, do not explain anything.
"""

    result = generator(prompt)[0]['generated_text']
    
    # Post-process (remove prompt if included in output)
    if prompt in result:
        result = result.split(prompt)[-1].strip()

    # Return only first sentence
    sentence = result.strip().split('\n')[0]
    
    return sentence

In [4]:
tokenizer.pad_token = tokenizer.eos_token

def batch_generate_mcqs(word_pos_cefr_list, language="English", batch_size=8, temperature=0.8, max_length=512):
    prompts = [
        f"""<s>[INST]
Your task is to generate a {language} multiple-choice question (MCQ) to test an {language} learner's understanding of a target word.

Instructions:
- The question should be suitable for a {language} language learner at the {cefr} CEFR level.
- The target word is "{word}" and its part of speech is {pos}.
- Test the **meaning** of the word in a realistic and understandable context.
- Make the questions and answers in {language} language only.
- Avoid using the word itself in the answer choices.
- Include 4 answer options: one correct and three plausible distractors.
- Distractors should be clearly incorrect but not obviously wrong.
- Mark the correct answer at the end.

Format:
Q: <question>
A) <option A>
B) <option B>
C) <option C>
D) <option D>
Answer: <correct letter>
[/INST]"""
        for word, pos, cefr in word_pos_cefr_list
    ]

    all_outputs = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i + batch_size]
        
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True)
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        
        outputs = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_outputs.extend(decoded)

    return all_outputs


In [5]:
def batch_generate_grammar_mcqs(grammar_rule_cefr_list, language, batch_size=8, temperature=0.8, max_length=512):
    prompts = [
       f"""<s>[INST]
Your task is to generate a multiple-choice question (MCQ) to test an {language} learner's understanding of a grammar rule.

Instructions:
- The question should be suitable for a learner at the {cefr} CEFR level.
- The questions and answers should always be in {language} language only.
- The grammar rule is: "{rule}".
- Use a realistic sentence context where applying the grammar rule is necessary.
- Avoid making the correct answer too obvious.
- Include 4 answer options:
    - One option must be grammatically correct according to the grammar rule: "{rule}" and fit the context of the question.
    - Three options must contain grammatical mistakes based on the context of the question and common errors expected at the user's level.
- Mark the correct answer at the end.

Format:
Q: <question>
A) <option A>
B) <option B>
C) <option C>
D) <option D>
Answer: <correct letter>
[/INST]"""
        for rule, cefr in grammar_rule_cefr_list
    ]

    all_outputs = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i + batch_size]
        
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True)
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        
        outputs = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_outputs.extend(decoded)

    return all_outputs  # ← raw strings only, not cleaned


In [6]:
def batch_generate_reading_exercises(requests, language, model, tokenizer, device="cuda", max_new_tokens=2048, batch_size=8, temperature=0.8, max_length=100000):
    prompts = [
        f"""<s>[INST]
Your task is to generate a reading comprehension exercise for an {language} learner.

Instructions:
- Write a short reading passage (~{req[1]} words) suitable for CEFR level {req[0]}.
- The topic of the passage should be "{req[2]}".
- The passage should be in {language}  only.
- Don't translate the passage.
- After the passage, generate 3 multiple-choice questions (MCQs) based on the passage.
- Each question should have 4 answer options (A–D), only one of which is correct.
- Those questions should be in {language} language.
- Each of the 3 questions MUST be followed by exactly one correct answer in the format: Answer: <letter>.
- Use clear and natural {language} suitable for learners.
- Avoid trick questions or overly complex sentence structures.
- don't ask questions that are unrelated to the passage or can't be deduced from it.
- don't ask about time of events if it's not explained in the passage.

Format:
Passage:
<your passage here>

Q1: <question>
A) <option A>
B) <option B>
C) <option C>
D) <option D>
Answer: <correct letter>
Q2: <question>
A) <option A>
B) <option B>
C) <option C>
D) <option D>
Answer: <correct letter>
Q3: <question>
A) <option A>
B) <option B>
C) <option C>
D) <option D>
Answer: <correct letter>
[/INST]"""
        for req in requests
    ]

    all_outputs = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True).to(device)
        outputs = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        )
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_outputs.extend(decoded)

    return all_outputs


In [7]:
def build_prompt(rule: str, user_info: dict, chat_history: str, language: str) -> str:
    user_name = user_info.get("name", "Student")
    level = user_info.get("level", "beginner")
    topics = user_info.get("topics", [])
    topics_text = ", ".join(topics) if topics else "general topics"

    prompt = f"""
You are a friendly and helpful {language} grammar tutor named "GrammarBot".

Your task:
- First, explain this grammar rule to {user_name}, who is a {level} learner.
- After the explanation, you MUST provide a natural example sentence related to the following topics: {topics_text}.
- You MUST ALWAYS include one example sentence.
- Write "Example:" and the sentence on next line.
- The example should be in {language} language.
- Explain the given {language} grammar rule in English based on its rules in {language}.
- IMPORTANT: Do NOT mention the user's interests, the topics, or why you chose the example.
- The example should sound natural — no labels like "(anime)", "(sports)" or similar.
- Always use clear, simple explanations.
- If this is the first message, always start with an explanation of the rule, then give the example.
- If the user asks something unrelated, gently remind them to stay on this rule.

GRAMMAR RULE:
{rule}

Conversation so far:
{chat_history}

Your next reply:
    """.strip()

    return prompt


In [8]:
def generate_grammar_reply(rule: str, user_info: dict , chat_history: str,
                           language: str = "English", temperature=0.7, max_length=1024) -> str:
    prompt = build_prompt(rule, user_info, chat_history, language)

    inputs = tokenizer([prompt], return_tensors="pt", padding=True)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        do_sample=True,
        temperature=temperature,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded[0]


In [9]:
def extract_next_reply(full_output):
    if "GrammarBot:" in full_output:
        reply = full_output.split("GrammarBot:")[-1].strip()

        if "Example:" in reply:
            parts = reply.split("Example:")
            explanation = parts[0].strip()
            example = "Example:" + parts[1].strip()
            return f"{explanation}\n{example}"
        else:
            # No example — return full reply anyway
            return reply
    else:
        return full_output.strip()


In [10]:
def update_chat_history(chat_history, speaker, message):
    chat_history = chat_history.strip()
    chat_history += f"\n {speaker}:\n{message}"
    return chat_history

In [11]:
rule_text = "Can/could/would you...? (requests, offers, permission). "
user_data = {
    "name": "Mohamed",
    "level": "intermediate",
    "topics": ["anime", "technology", "sports"]
}
language = "English"
chat_string=""
if chat_string=="":
    first_user_msg = f'can you explain to me the rule about "{rule_text}" and give examples?'
    chat_string = update_chat_history(chat_string, user_data["name"], first_user_msg)

# STEP 2: Generate model reply
full_output = generate_grammar_reply(rule_text, user_data, chat_string, language)

# STEP 3: Extract "Your next reply"
bot_reply = extract_next_reply(full_output)

# STEP 4: Add bot reply to chat history
chat_string = chat_string.strip() + f"GrammarBot:\n {bot_reply}"


print(chat_string)

Mohamed:
can you explain to me the rule about "Can/could/would you...? (requests, offers, permission). " and give examples?GrammarBot:
 Sure, I'd be happy to explain this rule to you. The "can/could/would you...?" structure is used to make requests, offer suggestions, or ask for permission. It's a polite way to express your needs and desires in a friendly and respectful manner.

Examples:

* Can you help me with my homework?
* Would you like to go for a movie tonight?
* Could I borrow your car for the weekend?

This is the end of the conversation, you don't need to provide more examples.


In [12]:
import requests
from bs4 import BeautifulSoup

def get_cambridge_definition(word, language):
    
    if language.lower() == "english":
        url = f"https://dictionary.cambridge.org/dictionary/english/{word}"
    elif language.lower() == "french":
        url = f"https://dictionary.cambridge.org/dictionary/french-english/{word}"
    else:
        return f"❌ Language '{language}' not supported."

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return f"❌ Definition not found for '{word}'."

    soup = BeautifulSoup(response.content, "html.parser")

    definitions = []

    if language.lower() == "english":
        for entry in soup.select(".def-block"):
            def_tag = entry.select_one(".def")
            if def_tag:
                definitions.append(def_tag.text.strip())
            if len(definitions) >= 3:
                break

    elif language.lower() == "french":
        for trans in soup.select(".def-body.ddef_b.ddef_b-t .trans.dtrans"):
            text = trans.text.strip()
            if text:
                definitions.append(text)
            if len(definitions) >= 5:
                break

    return definitions if definitions else f"❌ No definitions found for '{word}' in '{language}'."


In [13]:
get_cambridge_definition("bonjour", "French")

['hello',
 'hello',
 'to say hello to sb',
 'good morning ,  good afternoon , good-day,  good evening ,  good night',
 'good morning']

In [14]:
from collections import deque
import google.generativeai as genai
import re
import json
from kaggle_secrets import UserSecretsClient


# --- 1. Setup Gemini API ---
secret_label = "GEMINI_KEY"
GEMINI_KEY = UserSecretsClient().get_secret(secret_label)
genai.configure(api_key=GEMINI_KEY)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")


In [15]:
# --- 2. System Context ---
def build_system_context(user_profile, reading_topic, language):
    MISTAKE_SUMMARY_JSON = '''{
      "mistake_summary": {
        "grammar": {
          "topics": "details",
          "count": 1
        }
        "spelling": {
          "topics": "details",
          "count": 1
        }
        "vocabulary": {
          "topics": "details",
          "count": 1
        }
      }
    }'''
    return (
        f"You are a friendly {language} language tutor helping an {user_profile['level']} level student named {user_profile['name']}. "
        f"You help them improve reading skills focused on {reading_topic}. "
        f"Their preferred topics are: {', '.join(user_profile['topics'])}. "
        f"make the conversation and error corrections about the {language} language"
        f"reply only in {language} unless you are correcting an error then explain it in English"
        f"Wait for the student’s reply before responding. "
        f"if the user make mistakes correct him/her."
        f"**Do not simulate the student’s response. Always wait for their reply.**"
        f"Have a natural conversation based on these topics. "
        f"Ask questions or make comments one at a time. "
        f"Correct language mistakes gently, label mistakes as: 'Mistake type: grammar', etc. "
        f"When the student types 'exit', respond ONLY with valid JSON mistake summary in this format:\n"
        f"{MISTAKE_SUMMARY_JSON}\n"
        f"Do NOT include any additional text. Only output JSON."
    )

In [16]:
!pip install fastapi uvicorn nest-asyncio
!pip install pyngrok --quiet  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from pydantic import BaseModel
from typing import List
from fastapi import FastAPI
import nest_asyncio
import uvicorn
from threading import Thread
import requests
import os
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient
from firebase_admin import credentials, firestore
import firebase_admin

In [18]:
nest_asyncio.apply()

# Init Firebase
cred = credentials.Certificate("/kaggle/input/firebase-key/key.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

secret_client = UserSecretsClient()
ngrok_token = secret_client.get_secret("NGROK_AUTHTOKEN")
ngrok.set_auth_token(ngrok_token)

tunnel = ngrok.connect(8000)
public_url = tunnel.public_url
print("🚀 Public FastAPI URL:", public_url)
db.collection("api_public_urls").document("fastapi_url").set({
    "url": str(public_url)
})

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello from notebook"}

def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run_api).start()

🚀 Public FastAPI URL: https://a10d-34-69-222-210.ngrok-free.app                                    


In [19]:
import re

def clean_mcq_output(output: str):
    # Only keep content after the last [/INST]
    if "[/INST]" in output:
        output = output.split("[/INST]")[-1].strip()
    
    # Extract question
    q_match = re.search(r"Q:\s*(.*?)\nA\)", output, re.DOTALL)
    question = q_match.group(1).strip() if q_match else ""

    # Extract options
    options = {}
    for letter in ['A', 'B', 'C', 'D']:
        opt_match = re.search(rf"{letter}\)\s*(.*?)\n", output)
        if opt_match:
            options[letter] = opt_match.group(1).strip()

    # Extract answer
    answer_match = re.search(r"Answer:\s*([A-D])", output)
    answer = answer_match.group(1).strip() if answer_match else ""

    return {
        "question": question,
        "options": options,
        "answer": answer
    }


def clean_reading_exercise(text):
    # Extract only the part after [/INST]
    if '[/INST]' in text:
        text = text.split('[/INST]', 1)[1].strip()

    # Now proceed to extract the actual passage
    passage_match = re.search(r"Passage:\s*(.+?)\n\s*Q1:", text, re.DOTALL)
    if not passage_match:
        return None
    passage = passage_match.group(1).strip()

    # Extract 3 question blocks with answers
    question_blocks = re.findall(
        r"(Q\d:.*?)\nAnswer:\s*([A-D])\)", text, re.DOTALL
    )
    if len(question_blocks) != 3:
        return None

    cleaned_questions = []

    for full_block, answer_letter in question_blocks:
        lines = full_block.strip().split("\n")
        question_text = lines[0].strip()
        options = {}
        for line in lines[1:]:
            match = re.match(r"([A-D])\)\s*(.*)", line.strip())
            if match:
                options[match.group(1)] = match.group(2).strip()

        cleaned_questions.append({
            "question": question_text,
            "options": options,
            "answer": answer_letter
        })

    return {
        "passage": passage,
        "questions": cleaned_questions
    }

In [20]:
import time


time.sleep(5)

INFO:     Started server process [31]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [21]:
class WordItem(BaseModel):
    word: str
    pos: str
    cefr: str  

class MCQRequest(BaseModel):
    items: List[WordItem]
    language: str


@app.post("/generate-word-mcqs")
def generate_word_mcqs(request: MCQRequest):
    words = [(w.word, w.pos, w.cefr) for w in request.items]

    raw_outputs = batch_generate_mcqs(word_pos_cefr_list = words, language = request.language) 

    structured_mcqs = []
    for output in raw_outputs:
        if isinstance(output, str):
            mcq = clean_mcq_output(output)
            if mcq.get("question"):
                structured_mcqs.append(mcq)

    return {"mcqs": structured_mcqs}
    

In [22]:
response = requests.post(
    "http://127.0.0.1:8000/generate-word-mcqs",
    json = {
    "items": [
        {"word": "courir", "pos": "verb", "cefr": "B1"},       # "run" in French
        {"word": "bonheur", "pos": "noun", "cefr": "B1"}      # "happiness" in French
    ],
    "language": "French"
}
)

print(response.status_code)
print(response.json())

INFO:     127.0.0.1:35446 - "POST /generate-word-mcqs HTTP/1.1" 200 OK
200
{'mcqs': [{'question': "Quelqu'un est en train de <la vente> à l'épicerie. Qui est-ce ?", 'options': {'A': 'Le directeur', 'B': 'Un employé', 'C': 'Un client', 'D': 'Une visiteuse'}, 'answer': 'C'}, {'question': 'Which of the following words best describes the feeling of happiness or elation?', 'options': {'A': 'Joie', 'B': 'Écstase', 'C': 'Plaisir', 'D': 'Contentement'}, 'answer': 'A'}]}


In [23]:
class GrammarRuleCEFR(BaseModel):
    rule: str
    cefr: str

class GrammarMCQRequest(BaseModel):
    items: List[GrammarRuleCEFR]
    language: str

@app.post("/generate-grammar-mcqs")
def generate_grammar_mcqs(request: GrammarMCQRequest):
    grammar_inputs = [(item.rule, item.cefr) for item in request.items]
    
    raw_outputs = batch_generate_grammar_mcqs(grammar_rule_cefr_list = grammar_inputs, language = request.language)

    cleaned_mcqs = [clean_mcq_output(output) for output in raw_outputs if isinstance(output, str)]

    return {"mcqs": cleaned_mcqs}


In [24]:
response = requests.post(
    "http://127.0.0.1:8000/generate-grammar-mcqs",
    json={
        "items": [
            {"rule": "present perfect", "cefr": "A1"},
            {"rule": "past simple", "cefr": "A2"}
        ], 
        "language": "English"
    }
)
print(response.json())

INFO:     127.0.0.1:56882 - "POST /generate-grammar-mcqs HTTP/1.1" 200 OK
{'mcqs': [{'question': 'John has just finished his job when Sarah called. What does "just finished" mean in this context?', 'options': {'A': 'John started his job and finished it recently.', 'B': 'John finished his job yesterday.', 'C': 'John is currently working on his job and called Sarah.', 'D': 'John started his job at the beginning of the day and finished it just now.'}, 'answer': 'D'}, {'question': 'Yesterday, Jane went to the store. She saw many people there. She bought some fruits and vegetables. On her way back home, she saw a beautiful sunset.', 'options': {'A': 'Yesterday, Jane goes to the store. She saw many people there. She bought some fruits and vegetables. On her way back home, she see a beautiful sunset.', 'B': 'Yesterday, Jane went to the store. She saw many people there. She bought some fruits and vegetables. On her way back home, she saw a beautiful sunset.', 'C': 'Yesterday, Jane went to the 

In [25]:
class CERFWCTopic(BaseModel):
    cefr_level: str
    word_count: int
    topic: str

class ReadingMCQRequest(BaseModel):
    items: List[CERFWCTopic]
    language: str

@app.post("/generate-reading-mcqs")
def generate_reading_mcqs(request: ReadingMCQRequest):
    requests = [(item.cefr_level, item.word_count, item.topic) for item in request.items]
    
    raw_outputs = batch_generate_reading_exercises(requests = requests, language = request.language, model = model, tokenizer = tokenizer)
    
    cleaned_outputs = []
    for output in raw_outputs:
        if isinstance(output, str):
            cleaned = clean_reading_exercise(output) 
            if cleaned:
                cleaned_outputs.append(cleaned)

    return {"readings": cleaned_outputs}

In [26]:
response = requests.post(
    "http://127.0.0.1:8000/generate-reading-mcqs",
    json={
        "items": [
            {"cefr_level": "A2", "word_count": 80, "topic": "daily routine"},
            {"cefr_level": "B1", "word_count": 100, "topic": "healthy eating"}
        ],
        "language": "French"
    }
)

print(response.status_code)
print(response.json())

INFO:     127.0.0.1:40608 - "POST /generate-reading-mcqs HTTP/1.1" 200 OK
200
{'readings': [{'passage': "Je m'apprête à commencer mon jour quotidien. Je me réveille plus tôt que d'ordinaire, vers 6 h 30. Je prends un petit déjeuner, où je bois du café et mange une baguette avec du jamon. Puis, je vais au travail. Je travaille en tant que journaliste. J'ai mon bureau à la rue Rivoli, dans le 1er arrondissement. Le matin, je écrit des articles pour le journal *Le Monde*. Je termine lejour en allant au cinéma avec mes amis. Ce soir, nous allons voir un film intéressant.", 'questions': [{'question': "Q1: Vers quelle heure se réveille l'auteur du passage ?", 'options': {'A': '6 h 00', 'B': '6 h 15', 'C': '6 h 30', 'D': '6 h 45'}, 'answer': 'C'}, {'question': "Q2: Où travaille l'auteur ?", 'options': {'A': 'Dans le 7e arrondissement', 'B': "À l'île de la Grande Jette", 'C': 'Au Palais des Congrès', 'D': 'À la rue Rivoli'}, 'answer': 'D'}, {'question': "Q3: Quelle activité fait le passage men

In [27]:
@app.get("/definition/{language}/{word}")
async def definition(language: str, word: str):
    definitions = get_cambridge_definition(word, language)
    if not definitions:
        return JSONResponse(status_code=404, content={"error": f"Definition not found for '{word}' in language '{language}'."})
    return {"word": word, "definitions": definitions}

In [28]:
language = "english"
word = "a.m./A.M./am/AM"

response = requests.get(f"http://127.0.0.1:8000/definition/{language}/{word}")
print(response.status_code)
print(response.json())

INFO:     127.0.0.1:47210 - "GET /definition/english/a.m./A.M./am/AM HTTP/1.1" 404 Not Found
404
{'detail': 'Not Found'}


In [29]:
class ExampleRequest(BaseModel):
    word: str
    pos: str
    user_data: dict
    language: str

In [30]:
@app.post("/exampleWordPOS")
async def example_post(request: ExampleRequest):
    example_sentence = generate_personalized_example(request.word, request.pos, request.user_data, request.language)
    return {
        "word": request.word,
        "pos": request.pos,
        "example_sentence": example_sentence
    }

In [31]:
response = requests.post(
    "http://127.0.0.1:8000/exampleWordPOS",
    json={
        "word": "Être",
        "pos": "verb",
        "user_data": {
            "name": "Mohamed",
            "level": "intermediate",
            "topics": ["anime", "technology", "sports"]
        },
        "language": "French"
    }
)
print(response.status_code)
print(response.json())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     127.0.0.1:47226 - "POST /exampleWordPOS HTTP/1.1" 200 OK
200
{'word': 'Être', 'pos': 'verb', 'example_sentence': '"Je suis un animateur."'}


In [32]:
class GrammarBotRequest(BaseModel):
    rule_text: str
    user_data: dict
    chat_string: str = ""
    language: str

In [33]:
@app.post("/grammarChatBot")
async def grammar_reply(request: GrammarBotRequest):
    chat_string = request.chat_string

    if chat_string == "":
        # First user message if chat is new
        first_user_msg = f'explain to me the rule about "{request.rule_text}" and give examples?'
        chat_string = update_chat_history(
            chat_string, 
            request.user_data["name"], 
            first_user_msg
        )

    full_output = generate_grammar_reply(
        rule=request.rule_text,
        user_info=request.user_data,
        chat_history=chat_string,
        language=request.language
    )

    bot_reply = extract_next_reply(full_output)

    # Update chat history with bot reply as well:
    chat_string = update_chat_history(
        chat_string, 
        "GrammarBot", 
        bot_reply
    )

    return { "chat_string": chat_string }

In [34]:
response = requests.post(
    "http://127.0.0.1:8000/grammarChatBot",
    json={
        "rule_text": "past tense.",
        "user_data": {
            "name": "Mohamed",
            "level": "intermediate",
            "topics": ["anime", "technology", "sports"]
        },
        "language": "French",
        "chat_string": ""  # or pass existing chat_string for continuation
    }
)
print(response.status_code)
print(response.json())

INFO:     127.0.0.1:47234 - "POST /grammarChatBot HTTP/1.1" 200 OK
200
{'chat_string': 'Mohamed:\nexplain to me the rule about "past tense." and give examples?\n GrammarBot:\nSure, I\'d be happy to explain the rule of past tense for you, Mohamed.\n\nIn French, past tense is used to describe actions that have already happened in the past. To form the past tense of a verb, you need to add the suffix "-ait" to the base form of the verb in most cases. Here\'s an example:\n\nJe parle (I speak)\nJe parlais (I spoke)\n\nIn the past tense, "parlais" means "spoke" and "ai" is the past tense of the verb "avoir" (to have).\nExample:Je suis allé au cinéma avec mes amis hier soir (I went to the cinema with my friends last night)\n\nI hope that helps! Let me know if you have any other questions.'}


In [35]:
class ChatBotRequest(BaseModel):
    message:  str
    chat_string: str
    user_profile: dict
    language: str = "English"
    reading_topic: str

In [36]:
@app.post("/chat")
async def chat(request: ChatBotRequest):

    message = request.message
    chat_history = request.chat_string
    user_profile = request.user_profile
    language = request.language or "English"
    reading_topic = request.reading_topic

    required_fields = ["name", "level", "topics"]
    if not all(field in user_profile for field in required_fields):
        return JSONResponse({"error": f"Missing fields in user_profile. Required: {required_fields}"}, status_code=400)

    if not message:
        return JSONResponse({"error": "Missing 'message'."}, status_code=400)

    system_prompt = build_system_context(user_profile, reading_topic, language)
    full_conversation = f"System: {system_prompt}\n{chat_history}\n{user_profile['name']}: {message}"

    # Call Gemini
    response = gemini_model.generate_content(contents=full_conversation)
    bot_reply = response.text.strip()

    updated_chat = f"{chat_history}\nUser: {message}\nBot: {bot_reply}"

    result = {
        "reply": None,
        "chat_string": updated_chat,
        "mistake_summary": None,
        "recommendations": None
    }

    if message.strip().lower().startswith("exit"):
        clean_json = re.search(r"\{[\s\S]*\}", bot_reply)
        if clean_json:
            try:
                mistake_summary = json.loads(clean_json.group())
            except json.JSONDecodeError:
                return JSONResponse({"error": "Failed to parse mistake summary JSON.", "raw_response": bot_reply}, status_code=500)
        else:
            return JSONResponse({"error": "No JSON found in response.", "raw_response": bot_reply}, status_code=500)
        # Get improvement recommendations
        rec_prompt = (
            f"Based on this mistake summary:\n{json.dumps(mistake_summary, indent=2)}\n"
            f"Give very short, concise and friendly recommendations in English for improving {language} skills."
        )
        rec_response = gemini_model.generate_content(contents=rec_prompt)
        recommendations = rec_response.text.strip()

        result["mistake_summary"] = mistake_summary
        result["recommendations"] = recommendations
        result["chat_string"] = ""

    else:
        result["reply"] = bot_reply
        result["chat_string"] = updated_chat


    return result

In [37]:
import requests
import json

# Your FastAPI endpoint (assuming running on port 8000)
url = "http://127.0.0.1:8000/chat"

# Prepare the payload
payload = {
    "message": "I want to practice reading.",
    "chat_string": "User: Hi\nBot: Hello! What would you like to practice today?",
    "reading_topic": "past tense",
    "language": "French",
    "user_profile": {
        "name": "Mohamed",
        "level": "A2",
        "topics": ["anime", "technology", "sports"]
    }
}

# Send the POST request
response = requests.post(url, json=payload)

# Print the bot's reply and updated chat history
print("Status Code:", response.status_code)
print("Response:", response.json())


INFO:     127.0.0.1:33474 - "POST /chat HTTP/1.1" 200 OK
Status Code: 200
Response: {'reply': "Parfait ! Puisque tu aimes l'anime, la technologie et le sport, je te propose un petit texte sur un de ces sujets au passé. Lequel préfères-tu pour commencer ?", 'chat_string': "User: Hi\nBot: Hello! What would you like to practice today?\nUser: I want to practice reading.\nBot: Parfait ! Puisque tu aimes l'anime, la technologie et le sport, je te propose un petit texte sur un de ces sujets au passé. Lequel préfères-tu pour commencer ?", 'mistake_summary': None, 'recommendations': None}


In [38]:
import requests
import json

# Your FastAPI endpoint (assuming running on port 8000)
url = "http://127.0.0.1:8000/chat"

# Prepare the payload
payload = {
    "message": "exit.",
    "chat_string": (
        "User: Hi\n"
        "Bot: Hello! What would you like to practice today?\n"
        "User: I want to practice reading.\n"
        "Bot: Très bien! Dis-moi, que veux-tu lire aujourd'hui?\n"
        "User: Je lire le livre hier.\n"   # Incorrect verb conjugation: should be "j'ai lu le livre hier."
        "Bot: D'accord, continuons. Parle-moi d'hier.\n"
        "User: Hier je allé au parc avec mes amis.\n"  # Incorrect: should be "je suis allé"
        "Bot: Super! Et qu'avez-vous fait au parc?\n"
        "User: Nous joue football et parler beaucoup.\n"  # Incorrect: "Nous avons joué", "parlé"
    ),
    "reading_topic": "past tense",
    "language": "French",
    "user_profile": {
        "name": "Mohamed",
        "level": "A2",
        "topics": ["sports", "daily life", "travel"]
    }
}

# Send the POST request
response = requests.post(url, json=payload)

# Print the bot's reply and updated chat history
print("Status Code:", response.status_code)
print("Response:", response.json())


INFO:     127.0.0.1:33490 - "POST /chat HTTP/1.1" 200 OK
Status Code: 200
Response: {'reply': None, 'chat_string': '', 'mistake_summary': {'mistake_summary': {'grammar': {'topics': 'Verb conjugation in past tense, agreement', 'count': 2}, 'spelling': {'topics': 'None', 'count': 0}, 'vocabulary': {'topics': 'None', 'count': 0}}}, 'recommendations': 'Okay, here are some quick tips to help boost your French:\n\n*   **Past Tense Power:** Brush up on your verb conjugations, especially in the past!\n*   **Agree to Agree:** Double-check that your verbs are agreeing with your subjects!\n\nBon courage!'}


In [39]:
class TTSRequest(BaseModel):
    text: str
    language: str = "english"

In [40]:
!pip install azure-cognitiveservices-speech


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 MB 42.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 11.2 MB/s eta 0:00:00


In [41]:
import azure.cognitiveservices.speech as speechsdk
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
import os
import uuid
from IPython.display import Audio, display

# Azure credentials from environment variables
AZURE_TTS_KEY = UserSecretsClient().get_secret("AZURE_TTS_KEY")
SPEECH_REGION = "eastus"

if not AZURE_TTS_KEY or not SPEECH_REGION:
    raise ValueError("AZURE_TTS_KEY and SPEECH_REGION environment variables are required.")

# Global speech config
speech_config = speechsdk.SpeechConfig(subscription=AZURE_TTS_KEY, region=SPEECH_REGION)
speech_config.speech_synthesis_voice_name = 'en-US-AvaMultilingualNeural'  # Default voice

@app.post("/textToSpeech")
async def text_to_speech(request: TTSRequest):
    """Generate speech audio from text and return the audio file."""
    
    # Create unique filename to avoid conflicts
    audio_filename = f"tts_{uuid.uuid4().hex}.wav"
    audio_config = speechsdk.audio.AudioOutputConfig(filename=audio_filename)

    # Optionally allow voice change based on language (example logic)
    if request.language == "english":
        speech_config.speech_synthesis_voice_name = 'en-US-AvaMultilingualNeural'
    # Add more language-voice mappings if needed
    elif request.language.lower() == "french":
        speech_config.speech_synthesis_voice_name = 'fr-FR-DeniseMultilingualNeural'

    else:
        raise HTTPException(status_code=400, detail=f"Unsupported language: {request.language}")
        
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

    result = speech_synthesizer.speak_text_async(request.text).get()

    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        return FileResponse(audio_filename, media_type="audio/wav", filename="speech.wav")

    else:
        error_msg = "Speech synthesis failed."
        if result.reason == speechsdk.ResultReason.Canceled:
            details = result.cancellation_details
            if details.reason == speechsdk.CancellationReason.Error:
                error_msg = f"Speech synthesis canceled. Details: {details.error_details}"
        raise HTTPException(status_code=500, detail=error_msg)


In [42]:
url = "http://127.0.0.1:8000/textToSpeech"
payload = { "text": "Hello, this is a test from Azure TTS.",
  "language": "english"}
response = requests.post(url, json=payload)
if response.status_code == 200:
    # Save audio file locally
    with open("output.wav", "wb") as f:
        f.write(response.content)

    # Play audio in notebook
    display(Audio("output.wav"))
else:
    print(f"❌ Request failed with status {response.status_code}: {response.text}")

INFO:     127.0.0.1:39496 - "POST /textToSpeech HTTP/1.1" 200 OK
